the purpose of this notebook was to ensure the files from the thredds server were the same on the local storage

In [12]:
from pathlib import Path

import numpy as np
import xarray as xr

import utils

In [10]:
def parse_time(time):
    d = "".join(str(time).split("T")[0].split("-"))
    t = "".join(str(time).split("T")[1].split(":")[:2])
    return d, t

In [14]:
source_path = Path(utils.CURRENT_NETCDF_DIR) / "source/NetCDF"

res = "1km"
time_start = np.datetime64("2020-08-01T01", "m")
time_end = np.datetime64("2020-08-14T13", "m")
interval = np.timedelta64(1, "h")

lat_sl = slice(32.11093, 32.73124)
lon_sl = slice(-117.565, -116.9924)

In [15]:
ds_total = None
curr_time = time_start
while curr_time <= time_end:
    d, t = parse_time(curr_time)
    filename = f"{d}{t}_hfr_uswc_{res}_rtv_uwls_SIO.nc"
    ds = xr.open_dataset(source_path / filename)
    ds = ds.sel(lat=lat_sl, lon=lon_sl)
    if ds_total is None:
        ds_total = ds
    else:
        ds_total = xr.concat([ds_total, ds], dim="time")
    curr_time += interval

In [17]:
thredds = Path(utils.CURRENT_NETCDF_DIR) / "west_coast_1km_hourly/tj_plume.nc"
thredds_ds = xr.open_dataset(thredds)

In [23]:
(np.nan_to_num(thredds_ds["u"]) == np.nan_to_num(ds_total["u"])).all()

True

### thredds data is not any different from the source data